- 외부 데이터 사용 가능
  - 각 날짜에 대한 날씨 데이터, 요일 추가 등

In [1]:
# 라이브러리 임포트
import os
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch import autograd
from torch.utils import data
from torch.optim import Adam

In [2]:
# 난수 생성기가 항상 일정한 값을 출력하게 하기 위해 seed 고정
random_seed = 2022
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
np.random.seed(random_seed)

## EDA

In [4]:
DATASET_PATH = 'data'
# DATASET_PATH = os.path.join('/workspace/YearDream/task03_time-series_traffic/data')
# os.path.join 두 경로를 조인해주는 것.

In [16]:
df_train = pd.read_csv(os.path.join(DATASET_PATH, 'train.csv'))  # 비식별화된 도로 35개 컬럼

In [22]:
df_train.shape

(3279, 37)

In [23]:
df_val = pd.read_csv(os.path.join(DATASET_PATH, 'validate.csv'))
df_val.shape

(336, 37)

In [24]:
df_test = pd.read_csv(os.path.join(DATASET_PATH, 'test.csv'))
df_test.shape

(336, 37)

## Dataloader
* 한 칼럼에 대한 7일(168행) 데이터를 input_data, 뒤따르는 7일 데이터를 output_data로 반환합니다.
  - 일주일 단위로 구분.
  - 기간 설정은 커스텀해봐도 좋음.
* 도로별 차이를 두지 않고 모든 도로를 동일한 타입의 데이터로 취급합니다.
* 모든 csv 파일의 마지막 168행은 예측해야하는 값이므로 input으로 들어가지 않습니다.

In [ ]:
class CustomDataset(data.Dataset):      
  # CustomDataset 만들 때 보통 torch.utils.data.Dataset 클래스의 상속 클래스 CustomDataset 클래스 생성. 
  # 상속 클래스 생성시 __init__, __getitem__, __len__함수는 기본적으로 정의해줘야 함.
    
    def __init__(self, root, seq_len, batch_size=64, phase='train'):      # 데이터 로드 단계에 사용될 여러 변수들을 'self.변수명'의 형태로 지정해두는 함수
        
        self.root = root      # CustomDataset 객체 생성 시 데이터 경로 앞부분(공통 부분)을 root로 입력받아 저장
        self.phase = phase      # CustomDataset 객체 생성 시 데이터 경로 뒷부분(train/validate/test)을 phase로 입력받아 저장
        self.label_path = os.path.join(self.root, self.phase + '.csv')      # 데이터 전체 경로 생성
        df = pd.read_csv(self.label_path)      # 생성한 데이터 전체 경로로부터 데이터 로드
        
        self.seq_len = seq_len * 24     # 일 단위 기간을 입력 받은 후 시간 단위 기간으로 변환(24시간)하여 저장  # 시계열 길이.
        self.batch_size = batch_size
        self.labels = {}
        
        timestamps = [(i, j) for (i, j) in zip(list(df['날짜']), list(df['시간']))]      # 날짜와 시간 정보가 튜플로 들어 있는 리스트 생성
        categories = df.columns.values.tolist()[2:]      # 도로명 column list 생성 (날짜, 시간 컬럼 제외)

        input_data = []
        output_data = []

        for t in range(len(timestamps)):
            temp_input_data = []
            temp_output_data = []

            for col in categories:
                road = df[col].tolist()
                inp = [float(i) for i in road[t:t+self.seq_len]]      # input 데이터 시계열 구간 설정
                outp = [float(j) for j in road[t+self.seq_len:t+2*self.seq_len]]      # output 데이터 시계열 구간 설정
                temp_input_data.append(inp) 
                temp_output_data.append(outp)

            input_data.append(temp_input_data)
            output_data.append(temp_output_data)
            
# input_data : [[첫번째 input 기간 동안의 첫번째 도로의 통행량 list, ..., 첫번째 input 기간 동안의 35번째 도로의 통행량 list], ...,
#               [마지막 input 기간 동안의 첫번째 도로의 통행량 list, ..., 마지막 input 기간 동안의 35번째 도로의 통행량 list]]
# output_data : [[첫번째 output 기간 동안의 첫번째 도로의 통행량 list, ..., 첫번째 output 기간 동안의 35번째 도로의 통행량 list], ...,
#                [마지막 output 기간 동안의 첫번째 도로의 통행량 list, ..., 마지막 output 기간 동안의 35번째 도로의 통행량 list]]
        
        self.labels['timestamp'] = timestamps
        self.labels['category'] = categories
        self.labels['input'] = input_data
        self.labels['output'] = output_data

    def __getitem__(self, index):      # index를 가지고 데이터를 하나씩 불러올 수 있게 하는 함수

#         데이터 내 index가 부여되는 형태

#                 | road_1    road_2    ...  road_35
#                -------------------------------------
#         time_1  | index_0   index_1   ...  index_34
#         time_2  | index_35  index_36  ...  index_69

        row = index // 35      # index를 35(도로수)로 나눈 몫  ex) 71//35 -> 2
        col = index % 35      # index를 35(도로수)로 나눈 나머지  ex) 71%35 -> 1

        timestamp = self.labels['timestamp'][row]      # (날짜, 시간) 튜플이 들어있는 list에서 row번째 시점에 해당하는 튜플
        category = self.labels['category'][col]      # 도로명 column list에서 col번째 도로에 해당하는 element
        
        # 특정 시점의 특정 도로 교통량 가져오기
        input_data = torch.tensor(self.labels['input'][row][col])      # input_data list에서, row번째 시점의 col번째 도로 교통량 정보

        if self.phase != 'test':
            output_data = torch.tensor(self.labels['output'][row][col])
        else:
            output_data = []

        return timestamp, category, (input_data, output_data)

    def __len__(self):      # getitem 함수를 통해 데이터를 불러오려면,전체 index 길이를 알아야 한다. 그 길이만큼 메모리 할당하기 때문.
        return (len(self.labels['timestamp']) - (self.seq_len * 2) + 1) * 35     # 특정 시점이 아닌 특정 기간을 하나의 data 단위로 설정하면, 전체 샘플 수는 감소함을 반영 



def data_loader(root, phase='train', batch_size=64, seq_len=7, drop_last=False):
    if phase == 'train':
        shuffle = True
    else:
        shuffle = False

    dataset = CustomDataset(root, seq_len, batch_size, phase)
    dataloader = data.DataLoader(dataset=dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last)

    return dataloader

## Model

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


class LSTMNet(nn.Module):      
  # torch.nn.Module 클래스의 상속 class LSTMNet class 생성. 
  # 상속 클래스 생성시 __init__, forward 함수는 기본적으로 정의해줘야 함.
    def __init__(self,
                 input_size=168,      # input 길이는 168시간(7일 X 24시간)
                 hidden_size=1024,
                 output_size=168,      # output 길이는 168시간(7일 X 24시간)
                 batch_size=64,
                 num_layers=3,
                 dropout=0,
                 batch_first=False):      # batch_first(default=False) : 배치 차원을 첫번째 차원으로 하여 데이터를 불러올 것인지 여부

        super(LSTMNet, self).__init__()

        self.hidden_size = hidden_size
        
        ##### Layer 1
        self.lstm1 = nn.LSTM(input_size,
                             hidden_size,
                             dropout=0.2,
                             num_layers=num_layers)

        ##### Layer 2
        self.lstm2 = nn.LSTM(hidden_size, 
                             hidden_size,
                             dropout=0.2,
                             num_layers=num_layers)

        ##### Finalize
        self.linear = nn.Linear(hidden_size, 
                                output_size)
        
        self.activation = nn.LeakyReLU(0.2)

        
    def forward(self, x, h_in, c_in):      # forward 함수를 정의하지만 직접 호출할 일은 없음. model = LSTMNet() 객체를 생성한 다음, 
                                           # predictions, (h2, c_2) = model(input_data, h_in, c_in) 형태로 return을 받으면 됨 : 2개 리턴하기 때문에 이런 모양
        
        # 파라미터 생성. 가장 처음 초기값으로 들아갈 부분. 가장 처음 이후로는 아웃풋값을 인풋값으로 받음. 
        h_in = nn.Parameter(h_in.type(dtype), requires_grad=True)      # gradient descent로 업데이트 되는(requires_grad=True), hidden state 초기값 파라미터 생성 
        c_in = nn.Parameter(c_in.type(dtype), requires_grad=True)      # gradient descent로 업데이트 되는(requires_grad=True), cell state 초기값 파라미터 생성

        # Layer 1
        lstm_out, (h_1, c_1) = self.lstm1(x, (h_in, c_in))
        lstm_out = self.activation(lstm_out)

        # Layer2
        lstm_out, (h_2, c_2) = self.lstm2(lstm_out, (h_1, c_1))
        lstm_out = self.activation(lstm_out)

        # Final
        predictions = self.linear(lstm_out)
        
        return predictions, (h_2, c_2)  # 튜플 형태까지해서 2개 리턴됨

In [ ]:
# 결과 파일과 모델 가중치 파일 저장을 위해 log 디렉토리 생성. 중요한 파일이 덮어씌워지지 않도록 주의
os.makedirs('log', exist_ok=True)      # log 폴더 생성, 이미 생성되었을 시 추가로 생성하지 않도록 exist_ok=True


def save_model(model_name, model, optimizer):      # 모델 가중치 파일 저장 함수
    state = {
        'model': model.state_dict(),  # 모델 파라미터
        'optimizer': optimizer.state_dict()
    }
    torch.save(state, os.path.join('log', model_name + '.pth'))
    print('model saved\n')  # 학습 시 val acc 향상되면 저장되었음을 보여주기


def load_model(model_name, model, optimizer=None):      # 모델 가중치 파일 로드 함수
    state = torch.load(os.path.join(model_name))
    model.load_state_dict(state['model'])
    if optimizer is not None:
        optimizer.load_state_dict(state['optimizer'])
    print('model loaded')

## Hyperparameters
실험하면서 변경해보기

In [ ]:
dtype = torch.float
num_epochs = 5
base_lr = 0.01
seq_len = 7

input_size = seq_len * 24
hidden_size = 1024
output_size = input_size
batch_size = 64
num_layers = 6  # 세로축으로 층 많이 쌓으면 그래디언트 베니싱 생길 수 있음을 주의

## Training Setting

In [ ]:
# model
model = LSTMNet(input_size=input_size,
                hidden_size=hidden_size,
                output_size=output_size,
                batch_size=batch_size,
                num_layers=num_layers)

model = model.to(device)

# loss function
criterion = nn.MSELoss()      # 플랫폼 상 채점은 RMSE, 즉 MSE에 root를 씌운 값이기 때문에 사실상 평가지표와 같은 Loss입니다.

# optimizer
optimizer = Adam(model.parameters(), lr=base_lr)      # optimizer로는 Adam이 가장 무난합니다. Adam을 쓰면 learning_rate를 따로 지정해주지 않아도 알아서 조정되므로 유용.

In [ ]:
# 모델 구성 확인
print(model)    # 렐루, 리키렐루 : 극적인 변화를 기대하긴 어렵다고? 엑티베이션 변화로는?

LSTMNet(
  (lstm1): LSTM(168, 1024, num_layers=6, dropout=0.2)
  (lstm2): LSTM(1024, 1024, num_layers=6, dropout=0.2)
  (linear): Linear(in_features=1024, out_features=168, bias=True)
  (activation): LeakyReLU(negative_slope=0.2)
)


In [ ]:
# get data loader
train_dataloader = data_loader(root=DATASET_PATH,
                               phase='train',
                               batch_size=batch_size,
                               seq_len=seq_len,
                               drop_last=True)

validate_dataloader = data_loader(root=DATASET_PATH,
                                  phase='validate',
                                  batch_size=1,
                                  seq_len=seq_len,
                                  drop_last=True)

## Train

In [ ]:
train_batch_loss = 0.0      # 400 batch마다 평균 training loss를 확인한 다음, train_batch_loss를 0으로 갱신해줄 것
train_epoch_loss = 0.0      # 1 epoch마다 평균 training loss를 확인한 다음, train_epoch_loss를 0으로 갱신해줄 것

valid_epoch_loss = 0.0      # 1 epoch마다 평균 validation loss를 확인한 다음, valid_epoch_loss를 0으로 갱신해줄 것
valid_min_epoch_loss = np.inf      # 초기 loss를 마이너스 무한대로 설정해두고, validation epoch loss가 낮아질 때마다 갱신해줄 것


for epoch in range(num_epochs):

    model.train()      # 모델을 train mode로 전환. train mode일 때만 적용되어야 하는 drop out 등이 적용될 수 있게 하기 위해 작성해주는 코드.
    # 있으나 없으나 상관없기 때문에 지정해두고 들어가는 걸 권장

    for iter_, sample in enumerate(train_dataloader):      # enumerate 함수를 통해 train_dataloader에서 'batch의 index'와 'batch'를 순서대로 호출

        # 히든, 셀 스테이트 인풋값 설정
        (h_in, c_in) = (torch.zeros(num_layers, batch_size, hidden_size, requires_grad=True).to(device),
                        torch.zeros(num_layers, batch_size, hidden_size, requires_grad=True).to(device))

        _, _, (input_data, output_data) = sample      # train_dataloader에서 불러온 sample은 [[날짜, 시간], [도로], [[input_data],[output_data]]]로 구성됨.
                                                      # 학습에는 [[input_data], [output_data]]만 사용
        
        input_data = input_data.unsqueeze(0).to(device)
        output_data = output_data.unsqueeze(0).to(device)

        pred, (h_in, c_in) = model(input_data, h_in, c_in)
        
        loss = criterion(pred, output_data)

        model.zero_grad()    # 파라미터 업데이트는 batch 단위로 이루어지기 때문에, 매 batch마다 gradient를 초기화해주어야 함 
        loss.backward()      # backpropagation : 로스로 백프로파 해주기.
        optimizer.step()      # 파라미터 업데이트 : 백프로파 이후 파라미터 업데이트.
        
        train_batch_loss += loss.item()
        train_epoch_loss += loss.item()

        if iter_ % 400 == 399:      # 400개의 batch마다 training Loss 출력
            print('Train Epoch: {:2} | Batch: {:4} | Loss: {:1.2f}'.format(epoch, iter_+1, train_batch_loss/400))
            train_batch_loss = 0
            
    train_epoch_loss = 0.0      # training epoch마다 train_epoch_loss 새로 구해줄 것

    ######################################################################
    ######################################################################

    model.eval()      # 모델을 eval mode로 전환. eval mode에서 적용되면 안되는 drop out 등이 적용되지 않게 하기 위함

    with torch.no_grad():      # validation / test set에 대해서는 weight 및 bias의 update, 즉, gradient descent가 일어나지 않도록 no_grad()를 선언
        (h_in, c_in) = (torch.zeros(num_layers, 1, hidden_size, requires_grad=False).to(device),
                        torch.zeros(num_layers, 1, hidden_size, requires_grad=False).to(device))

        for iter_, sample in enumerate(validate_dataloader):      # enumerate 함수를 통해 validate_dataloader에서 'batch의 index'와 'batch'를 순서대로 호출

            _, _, (input_data, output_data) = sample      # validate_dataloader에서 불러온 sample은 [[날짜, 시간], [도로], [[input_data],[output_data]]]로 구성됨. validation에는 [[input_data], [output_data]]만 사용

            input_data = input_data.unsqueeze(0).to(device)
            output_data = output_data.unsqueeze(0).to(device)

            pred, (h_in, c_in) = model(input_data, h_in, c_in)
            loss = criterion(pred, output_data)
            valid_epoch_loss += loss.item()

        print('\nValid Epoch: {:2} | Loss: {:1.2f}'.format(epoch, valid_epoch_loss/len(validate_dataloader)))

        if valid_epoch_loss < valid_min_epoch_loss:  # 최저값과 비교해서 갱신
            save_model('best', model, optimizer)
            valid_min_epoch_loss = valid_epoch_loss

        valid_epoch_loss = 0.0

        # val에서는 에폭마다 결과 출력

Train Epoch:  0 | Batch:  400 | Loss: 3729027214.72
Train Epoch:  0 | Batch:  800 | Loss: 3536851884.64
Train Epoch:  0 | Batch: 1200 | Loss: 3583481488.32
Train Epoch:  0 | Batch: 1600 | Loss: 3574592618.56

Valid Epoch:  0 | Loss: 3658084229.46
model saved

Train Epoch:  1 | Batch:  400 | Loss: 3472335433.60
Train Epoch:  1 | Batch:  800 | Loss: 3441780379.36
Train Epoch:  1 | Batch: 1200 | Loss: 3317598960.16
Train Epoch:  1 | Batch: 1600 | Loss: 3339269785.28

Valid Epoch:  1 | Loss: 3429849594.74
model saved

Train Epoch:  2 | Batch:  400 | Loss: 3383182452.64
Train Epoch:  2 | Batch:  800 | Loss: 3201422413.52
Train Epoch:  2 | Batch: 1200 | Loss: 3128741174.88
Train Epoch:  2 | Batch: 1600 | Loss: 3108163185.44

Valid Epoch:  2 | Loss: 3251222053.37
model saved

Train Epoch:  3 | Batch:  400 | Loss: 3098701018.08
Train Epoch:  3 | Batch:  800 | Loss: 3126002511.20
Train Epoch:  3 | Batch: 1200 | Loss: 3056091947.44
Train Epoch:  3 | Batch: 1600 | Loss: 2939116806.08

Valid Epoch

## Inference

In [ ]:
dtype = torch.float
seq_len = 7

input_size = seq_len * 24
hidden_size = 1024
output_size = input_size
batch_size = 1
num_layers = 6

In [ ]:
test_dataloader = data_loader(root=DATASET_PATH,
                              phase='test',
                              batch_size=batch_size,
                              seq_len=seq_len,
                              drop_last=True)

In [ ]:
model = LSTMNet(input_size=input_size,
                hidden_size=hidden_size,
                output_size=output_size,
                batch_size=batch_size,
                num_layers=num_layers)

# model
model_name = 'log/best.pth'  # 최고 모델 불러오기

load_model(model_name, model)
model = model.to(device)

model loaded


In [ ]:
submission_file_path = os.path.join(DATASET_PATH, 'sample_submission.csv')
submission_table = pd.read_csv(submission_file_path)

In [ ]:
(h_in, c_in) = (torch.zeros(num_layers, 1, hidden_size, requires_grad=False).to(device),
                torch.zeros(num_layers, 1, hidden_size, requires_grad=False).to(device))

for iter_, sample in enumerate(test_dataloader):

    timestamp, category, (input_data, output_data) = sample
    input_data = input_data.unsqueeze(0).to(device)

    pred, (h_in, c_in) = model(input_data, h_in, c_in)

    for i, (t, h) in enumerate(zip(timestamp[0], timestamp[1])):
        for cat, row in zip(category, pred[0]):
            cat = f'{cat}'
            submission_table[cat] = row.tolist()

NameError: name 'test_dataloader' is not defined

In [ ]:
submission_table.to_csv('prediction.csv', index=False)

In [3]:
import os
os.getcwd()

'/USER/traffic-volume'